In [6]:
from __future__ import print_function
import cv2
import numpy as np

In [7]:
#最多自動取幾個keypoints
MAX_FEATURES = 500
#取得的keypoints都有分數，設定程式僅取最佳的前15%
GOOD_MATCH_PERCENT = 0.15

In [8]:
#進行兩張相片的拼接
def alignImages(im1, im2):
    # 轉為灰階
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # 偵測 ORB keypoints及取得descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    im1Keypoints = np.array([])
    # cv2.drawKeypoints的參數有五個：原始圖片、keypoints、輸出的結果圖片、繪製顏色、繪圖功能設定。
    im1Keypoints = cv2.drawKeypoints(image1, keypoints1, im1Keypoints, color=(0,0,255),flags=0)
    print("Saving Image with Keypoints"); 
    cv2.imwrite("keypoints.jpg", im1Keypoints)

    # 特徵匹配.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    print(descriptors1, descriptors2)
    matches = matcher.match(descriptors1, descriptors2, None)

    # 每個匹配到的特徵帶有分數，由小至大排序
    matches.sort(key=lambda x: x.distance, reverse=False)

    # 僅保留前15%分數較高的匹配特徵
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # 繪出兩張相片匹配的特徵點
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    print("Saving Image with matches"); 
    cv2.imwrite("matches.jpg", imMatches)

    # 分別將兩張相片匹配的特徵點匯出
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # 找到homography。
    h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)

    # 套用homography
    im1Height, im1Width, channels = im1.shape
    im2Height, im2Width, channels = im2.shape

    im2Aligned = cv2.warpPerspective(im2, h, (im2Width + im1Width, im2Height))

    # 將image1圖像置換到對齊好的圖片中
    stitchedImage = np.copy(im2Aligned)
    stitchedImage[0:im1Height,0:im1Width] = image1

    return im2Aligned, stitchedImage

In [9]:
imageFile1 = "D:\Research\opencv\image\cam4_undistored.jpg"
print("Reading First image : ", imageFile1)
image1 = cv2.imread(imageFile1, cv2.IMREAD_COLOR)
cv2.imshow("First image ",image1)
cv2.waitKey(0)

imageFile2 = "D:\Research\opencv\image\cam3_undistored.jpg"
print("Reading Second Image : ", imageFile2);  
image2 = cv2.imread(imageFile2, cv2.IMREAD_COLOR)

print("Aligning images ...")
im2Aligned, stitchedImage = alignImages(image1, image2)
print("Saving aligned image"); 
cv2.imwrite("aligned-image.jpg", im2Aligned)
cv2.imshow("aligned image ", im2Aligned)

print("Saving stitched image"); 
cv2.imwrite("stitched-image.jpg", stitchedImage)
cv2.imshow("stitched image ", stitchedImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

Reading First image :  D:\Research\opencv\image\cam4_undistored.jpg
Reading Second Image :  D:\Research\opencv\image\cam3_undistored.jpg
Aligning images ...
Saving Image with Keypoints
[[249  90 255 ... 176 110 121]
 [ 72 185 189 ...  80 135 107]
 [  1 143 247 ... 219 224 173]
 ...
 [238 176 246 ... 213 199  83]
 [  5  82 191 ... 243  65 161]
 [248  32 146 ... 128  66  34]] [[ 41  33 122 ...  14  91 142]
 [173 100 196 ... 255 202 132]
 [253 255 232 ... 151 222  93]
 ...
 [198 248 149 ...  85   7 247]
 [148  84 255 ... 249 245 115]
 [  3 188 135 ...  68  35 156]]
Saving Image with matches
Saving aligned image
Saving stitched image


In [2]:
image_paths=["D:\Research\opencv\image\cam3.jpg", "D:\Research\opencv\image\cam4.jpg"]
# initialized a list of images
imgs = []

for i in range(len(image_paths)):
    imgs.append(cv2.imread(image_paths[i]))
    imgs[i]=cv2.resize(imgs[i],(0,0),fx=0.4,fy=0.4)

# showing the original pictures
#cv2.imshow('1',imgs[0])
#cv2.imshow('2',imgs[1])
#cv2.imshow('3',imgs[2])
#cv2.imshow('4',imgs[3])
#cv2.waitKey()
#cv2.destroyAllWindows()

In [3]:

stitchy=cv2.Stitcher.create()
(dummy,output)=stitchy.stitch(imgs)
 
if dummy != cv2.STITCHER_OK:
  # checking if the stitching procedure is successful
  # .stitch() function returns a true value if stitching is
  # done successfully
    print("stitching ain't successful")
else:
    print('Your Panorama is ready!!!')
    cv2.imshow('final result',output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


stitching ain't successful
